In [1]:
from transformers import AutoTokenizer, AutoModel
import torch 
import pandas as pd
from tqdm import tqdm 
import numpy as np

In [2]:

tokenizer = AutoTokenizer.from_pretrained('mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es')
model = AutoModel.from_pretrained('mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es')

#Creamos el texto y tokenizamos 
text = ["Esta es una reseña de una película. Mision imposible es una película genial"]
inputs = tokenizer(text, return_tensors='pt')

#Ahora vemos los tokens
tokens = inputs['input_ids'][0]
print("Tokens:", tokens)

#Ahora convertimos los tokens a palabras para comprobar que este correcto
decoded_text = tokenizer.decode(tokens)
print("Decoded Text", decoded_text)


Tokens: tensor([    4,  1359,  1058,  1108, 22181,  1532,  1008,  1108,  4264,  8186,
         1009, 16696,  6163,  1058,  1108,  4264,  8186,  4949,     5])
Decoded Text [CLS] esta es una resena de una pelicula. mision imposible es una pelicula genial [SEP]


In [3]:
def get_distilBert_embeddings(text):
    #Tokenizamos el texto
    inputs = tokenizer(text, return_tensors='pt')

    #Generamos los embeddings enviando el input al modelo 
    with torch.no_grad():
        outputs = model(**inputs)
    
    #Promediamos los embeddings a lo largo de la secuencia
        #Extraemo los embeddings
    embeddings = outputs.last_hidden_state
    pooled_embeddings = torch.mean(embeddings, dim=1)
    return pooled_embeddings.numpy()[0]

In [4]:

text = ["Esta es una reseña de una película. Mision imposible es una película genial"]

In [5]:
embeddings_dict = {}

for word in tqdm(text):
    #Generamos el embedding con la función get_bert_embeddings
    embedding = get_distilBert_embeddings(word)

    #Guardamos el embedding en el diccionario
    if len(embedding) == 768:
        embeddings_dict[word] = embedding

100%|██████████| 1/1 [00:00<00:00,  1.44it/s]


In [6]:
# Guardar las palabras en un CSV
pd.DataFrame(embeddings_dict.keys()).to_csv("words_bert.csv", index=False, columns=None, sep="\t")

# Guardar los embeddings en otro CSV
pd.DataFrame(embeddings_dict.values()).to_csv("embeddings_bert.csv", index=False, columns=None, sep="\t")